In [1]:
with open("../data/the_verdict.txt","r") as f:
    raw_text=f.read()

In [2]:
import re

In [3]:
example="Hello,My name is Aryan?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', example)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'My', 'name', 'is', 'Aryan', '?']


In [10]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
preprocessed = [item.lower() for item in preprocessed]
print(len(preprocessed))
print(preprocessed[:30])

4690
4690
['i', 'had', 'always', 'thought', 'jack', 'gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [ ]:
from collections import defaultdict, Counter

def get_stats(vocab):
    """Get frequency of adjacent symbol pairs in the vocabulary."""
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[symbols[i], symbols[i + 1]] += freq
    return pairs

def merge_vocab(pair, vocab_in):
    """Merge the most frequent pair in the vocabulary."""
    vocab_out = {}
    bigram = ' '.join(pair)
    print(f"Bigram: {bigram}")
    replacement = ''.join(pair)
    print(f"replacement: {replacement}")
    for word in vocab_in:
        word_out = word.replace(bigram, replacement)
        vocab_out[word_out] = vocab_in[word]
    return vocab_out

def bpe_algorithm(text_corpus, num_merges):
    """
    Main BPE algorithm implementation.
    
    Args:
        text_corpus: List of preprocessed words
        num_merges: Number of BPE merge operations to perform
    
    Returns:
        BPE vocabulary and merge operations
    """
    # Step 1: Initialize vocabulary with character-level tokens
    vocab = Counter(text_corpus)
    
    print(len(vocab))
    # Convert words to character sequences with end-of-word token
    char_vocab = defaultdict(int)
    for word, freq in vocab.items():
        # Add space between characters and </w> at the end
        chars = ' '.join(list(word)) + ' </w>'
        char_vocab[chars] = freq
    print(char_vocab)

    vocab = dict(char_vocab)
    merges = {}  # Store merge operations
    
    # # Step 2: Perform BPE merges

    for i in range(num_merges):
        # Get frequencies of all adjacent pairs
        pairs = get_stats(vocab)
        
        if not pairs:
            break
            
    #     # Find the most frequent pair
        best_pair = max(pairs, key=pairs.get)
        best_freq = pairs[best_pair]
        
        if best_freq <= 1:  # Stop if no frequent pairs
            break

        print(best_pair)
        print(best_freq)

        
        
    #     # Store merge operation
        merges[best_pair] = i
        
    #     # Merge the pair in vocabulary
    #     vocab = merge_vocab(best_pair, vocab)
        
    #     print(f"Iteration {i+1}: Merge {best_pair} (frequency: {best_freq})")
    
    # return vocab, merges


In [23]:
vocab,merge=bpe_algorithm(preprocessed,1000)

1076
defaultdict(<class 'int'>, {'i </w>': 122, 'h a d </w>': 55, 'a l w a y s </w>': 6, 't h o u g h t </w>': 8, 'j a c k </w>': 15, 'g i s b u r n </w>': 19, 'r a t h e r </w>': 6, 'a </w>': 83, 'c h e a p </w>': 1, 'g e n i u s </w>': 1, '- - </w>': 97, 't h o u g h </w>': 3, 'g o o d </w>': 2, 'f e l l o w </w>': 2, 'e n o u g h </w>': 5, 's o </w>': 16, 'i t </w>': 83, 'w a s </w>': 72, 'n o </w>': 9, 'g r e a t </w>': 2, 's u r p r i s e </w>': 3, 't o </w>': 100, 'm e </w>': 37, 'h e a r </w>': 4, 't h a t </w>': 60, ', </w>': 229, 'i n </w>': 41, 't h e </w>': 184, 'h e i g h t </w>': 2, 'o f </w>': 103, 'h i s </w>': 65, 'g l o r y </w>': 3, 'h e </w>': 79, 'd r o p p e d </w>': 2, 'p a i n t i n g </w>': 9, 'm a r r i e d </w>': 3, 'r i c h </w>': 2, 'w i d o w </w>': 1, 'a n d </w>': 84, 'e s t a b l i s h e d </w>': 1, 'h i m s e l f </w>': 9, 'v i l l a </w>': 1, 'o n </w>': 28, 'r i v i e r a </w>': 3, '. </w>': 212, '( </w>': 3, 'w o u l d </w>': 9, 'h a v e </w>': 26, '

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
merges

In [164]:
len(merge)

736

In [167]:
merge

{('e', '</w>'): 0,
 ('t', '</w>'): 1,
 ('d', '</w>'): 2,
 ('s', '</w>'): 3,
 ('t', 'h'): 4,
 ('n', '</w>'): 5,
 ('y', '</w>'): 6,
 ('i', 'n'): 7,
 (',', '</w>'): 8,
 ('.', '</w>'): 9,
 ('o', 'u'): 10,
 ('e', 'r'): 11,
 ('th', 'e</w>'): 12,
 ('e', 'd</w>'): 13,
 ('a', 'n'): 14,
 ('o', '</w>'): 15,
 ('f', '</w>'): 16,
 ('"', '</w>'): 17,
 ('h', 'a'): 18,
 ('in', 'g'): 19,
 ('h', 'i'): 20,
 ('i', '</w>'): 21,
 ('o', 'n'): 22,
 ('h', 'e</w>'): 23,
 ('t', 'o</w>'): 24,
 ('s', 't'): 25,
 ('o', 'f</w>'): 26,
 ('w', 'a'): 27,
 ('o', 'r'): 28,
 ('-', '-'): 29,
 ('--', '</w>'): 30,
 ('a', '</w>'): 31,
 ("'", '</w>'): 32,
 ('ing', '</w>'): 33,
 ('i', 't</w>'): 34,
 ('an', 'd</w>'): 35,
 ('e', 'a'): 36,
 ('e', 'n'): 37,
 ('er', '</w>'): 38,
 ('u', 'r'): 39,
 ('e', 'l'): 40,
 ('wa', 's</w>'): 41,
 ('m', '</w>'): 42,
 ('o', 'w'): 43,
 ('hi', 's</w>'): 44,
 ('a', 'c'): 45,
 ('a', 'l'): 46,
 ('t', 'i'): 47,
 ('tha', 't</w>'): 48,
 ('m', 'e</w>'): 49,
 ('ha', 'd</w>'): 50,
 ('h', '</w>'): 51,
 ('b', 'e